In [1]:
from pynwb import NWBHDF5IO
import numpy as np

# Path to your NWB file
file_path = '/Users/XinNiuAdmin/Documents/NWBTest/output/Screening/569_Screening/Experiment2/nwb/ecephys.nwb'

raw_io = NWBHDF5IO(file_path, "r")
nwb_in = raw_io.read()


/opt/miniconda3/envs/nwbPipeline/lib/python3.8/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/opt/miniconda3/envs/nwbPipeline/lib/python3.8/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/opt/miniconda3/envs/nwbPipeline/lib/python3.8/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)
/opt/miniconda3/envs/nwbPipeline/lib/python3.8/site-packages/pynwb/ecephys.py:90: UserWarning: ElectricalSeries 'ElectricalSeries': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


In [2]:
nwb_in

,x,y,z,location,group,group_name,label
id,,,,,,,
0,111.0,111.0,111.0,GB1-ROF1,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF1
1,111.0,111.0,111.0,GB1-ROF2,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF2
2,111.0,111.0,111.0,GB1-ROF3,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF3
3,111.0,111.0,111.0,GB1-ROF4,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF4
,x,y,z,location,group,group_name,label
id,,,,,,,
0,111.0,111.0,111.0,GB1-ROF1,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF1
1,111.0,111.0,111.0,GB1-ROF2,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF2
2,111.0,111.0,111.0,GB1-ROF3,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF3


In [3]:
nwb_in.electrodes

,x,y,z,location,group,group_name,label
id,,,,,,,
0,111.0,111.0,111.0,GB1-ROF1,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF1
1,111.0,111.0,111.0,GB1-ROF2,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF2
2,111.0,111.0,111.0,GB1-ROF3,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF3
3,111.0,111.0,111.0,GB1-ROF4,GB1 pynwb.ecephys.ElectrodeGroup at 0x4808406208\nFields:\n description: electrode group for GB1\n device: array pynwb.device.Device at 0x4808215568\nFields:\n description: Neuralynx Pegasus\n manufacturer: Neuralynx\n\n location: ROF\n,GB1,GB1-ROF4


In [33]:
# load LFP:
import numpy as np

np.array(nwb_in.processing['ecephys']['LFP']['ElectricalSeries'].data)

array([[-18.40234243,  10.80322328,  84.32062231, -43.67105949,
         11.26099281],
       [-18.37153093,  17.7003941 ,  94.56497972, -25.58968652,
         19.84044154],
       [-12.03593971,  16.18467544,  97.11259952, -21.18286781,
         22.89461368],
       ...,
       [-53.97800081, -65.5086647 ,  50.09895734,  10.05538199,
         -7.467085  ],
       [-54.81386844, -64.19449003,  51.80142303,   8.03404753,
         -8.4464247 ],
       [-56.12349011, -63.82082406,  53.7103421 ,   6.67310869,
         -3.00778042]])

In [35]:
# load spikes:
display(nwb_in.units['spike_times'][4])
display(nwb_in.units['electrodes'][4])

array([1.70620203e+09, 1.70620203e+09, 1.70620203e+09, 1.70620209e+09,
       1.70620209e+09, 1.70620209e+09, 1.70620209e+09, 1.70620209e+09,
       1.70620209e+09, 1.70620209e+09, 1.70620209e+09, 1.70620209e+09,
       1.70620209e+09, 1.70620209e+09, 1.70620210e+09, 1.70620214e+09,
       1.70620219e+09, 1.70620220e+09, 1.70620220e+09, 1.70620220e+09,
       1.70620220e+09, 1.70620232e+09, 1.70620243e+09, 1.70620243e+09,
       1.70620243e+09, 1.70620286e+09, 1.70620302e+09, 1.70620307e+09,
       1.70620310e+09, 1.70620310e+09, 1.70620310e+09, 1.70620310e+09,
       1.70620310e+09, 1.70620310e+09, 1.70620310e+09, 1.70620310e+09,
       1.70620310e+09, 1.70620310e+09, 1.70620310e+09, 1.70620316e+09,
       1.70620339e+09, 1.70620339e+09, 1.70620339e+09, 1.70620339e+09,
       1.70620339e+09, 1.70620339e+09, 1.70620339e+09, 1.70620339e+09,
       1.70620339e+09, 1.70620339e+09, 1.70620339e+09, 1.70620350e+09,
       1.70620350e+09, 1.70620350e+09, 1.70620350e+09])

,x,y,z,location,group,group_name,label
id,,,,,,,
2,111.0,111.0,111.0,GB1-ROF3,GB1 pynwb.ecephys.ElectrodeGroup at 0x48084062...,GB1,GB1-ROF3


In [11]:
# load average waveforms:
nwb_in.units['waveform_mean'][4]

array([  7.94588238,   7.22972122,   6.32011264,   5.24323231,
         4.03986067,   2.77641434,   1.54865704,   0.47389325,
        -0.33060262,  -0.78065453,  -0.86274151,  -0.66505179,
        -0.389336  ,  -0.33279791,  -0.83552745,  -2.19896154,
        -4.59218196,  -7.97167438, -12.04239023, -16.28088912,
       -20.02540364, -22.61694411, -23.55690824, -22.63721145,
       -20.00320085, -16.12748822, -11.69905001,  -7.45797214,
        -4.02277455,  -1.75865518,  -0.72079858,  -0.68259305,
        -1.23270196,  -1.90611845,  -2.30815579,  -2.19774389,
        -1.51347114,  -0.34569937,   1.12612546,   2.70467376,
         4.23044274,   5.60765266,   6.79873319,   7.79711007,
         8.59411841,   9.15563822,   9.41829533,   9.30657873,
         8.76398059,   7.78598354,   6.4419305 ,   4.87662624,
         3.28936274,   1.89546946,   0.88091234,   0.36238825,
         0.36347792,   0.81255114,   1.56197156,   2.42266762,
         3.20485536,   3.75514433,   3.98206652,   3.86